# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [4]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [4]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [5]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh8vLApAICIAALRVUQAAAAAY","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [6]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [7]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [8]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [13]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   More generally, content analysis is research u...
7   By having contents of communication available ...
8   Robert Weber notes: "To make valid inferences ...
9   There are five types of texts in content analy...
10  Over the years, content analysis has been appl...
11  In recent times, particularly with the advent ...
12  Quantitative content analysis has enjoyed a re...
13  Content analysis can also be described as stud...
14  Holsti groups fifteen uses of content analysis...
15  He also places these uses into the context of ...
16  The following table shows fifteen uses of cont...
17  The process of the initi

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [14]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [15]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [16]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [168]:
# Find out the list of most followed instagram accounts from https://www.trackalytics.com/the-most-followed-instagram-profiles/page/1/
# As a test, only the first page was scraped here.

my_url = "https://www.trackalytics.com/the-most-followed-instagram-profiles/page/1/"
my_request = requests.get(my_url)
my_soup = bs4.BeautifulSoup(my_request.text)
print(my_soup.text[:1000])

 



The Most Followed Instagram Profiles | Trackalytics













Menu 



  TRACKALYTICS


 Facebook



                Find a Fanpage:




Popular Statistics

The Most Liked NFL Teams 
The Most Liked NBA Teams 
The Most Liked MLB Teams 


Today Statistics

Todays Most Liked Facebook Fanpages
Todays Most Talked About Facebook Fanpages


All-time Statistics

The Most Liked Pages
The Most Talked About Pages 
The Most Checked In Pages 
The Most Liked by Category

Actor
Actor/director
Advertising Agency
Airport
Album
App page
Art
Artist
Arts & Entertainment
Arts & Humanities Website
Arts/entertainment/nightlife
Athlete
Attractions/things to do
Author
Automobiles and parts
Automotive
Baby Goods/Kids Goods
Band
Bank/financial institution
Blogger
Brand
Broadcasting & Media Production Company
Cars
Cause
Clothing
Clothing (Brand)
Clothing Store
College & University
Comedian
Community
Community organization
Company
Computer Company
Computers/internet website
Computers/technology
Consulting 

In [169]:
# Aquiring the head of the table
url_head = my_soup.thead.findAll('th')
my_head = []
for attr in url_head:
    # remove the (today) label from the table head
    my_head.append(re.sub(r'\(today\)', '', attr.text))
print (my_head)

['Rank', 'Profile', 'Total Followers', 'Total Following', 'Total Posts', 'Total Influence']


In [170]:
# Aquiring the items of the table
url_rows = my_soup.tbody.findAll('tr')
my_rows = []
for row in url_rows:
    # remove the "| Tweet" label from the table head
    my_rows.append(re.sub(r'\| Tweet', '',row.text))
print (my_rows[:3])

['\n\n\n1\n\n\n \n\n\nInstagram\n\n                  327,917,534                  \n                                        ( 155,955)\n                                        \n\n                  224                  \n                                        ( 1)\n                                        \n\n                  6,192                  \n                                        ( 1)\n                                        \n\n                  0.27%\n                  \n                                        (0.00%)\n                                        \n', '\n\n\n2\n\n\n \n\n\nCristiano Ronaldo\n\n                  198,127,767                  \n                                        ( 132,914)\n                                        \n\n                  444                  \n                                        (0)\n                                        \n\n                  2,742                  \n                                        (0)\n            

In [171]:
for i in range(0,len(my_rows)):
    # Removing the commas within numbers
    my_rows[i] = re.sub(r',','',my_rows[i])
    # Removing the whitespace within in brackets
    my_rows[i] = re.sub(r'\( ','(',my_rows[i])
    number = len(re.findall(r'\w \w', my_rows[i])) + 1
    my_rows[i] = [x for x in my_rows[i].split() if '(' not in x]
    my_rows[i][1] = ' '.join([my_rows[i][1+j] for j in range(0,number)])
    del my_rows[i][2:1 + number]
print (my_rows)

[['1', 'Instagram', '327917534', '224', '6192', '0.27%'], ['2', 'Cristiano Ronaldo', '198127767', '444', '2742', '3.46%'], ['3', 'Ariana Grande', '171750334', '999', '4477', '1.06%'], ['4', 'therock', '168535736', '342', '4740', '1.04%'], ['5', 'Selena Gomez', '166119117', '92', '1573', '2.82%'], ['6', 'Kylie', '158087797', '134', '6274', '4.06%'], ['7', 'Kim Kardashian West', '157248175', '14', '4976', '0.91%'], ['8', 'Leo Messi', '140974925', '227', '556', '2.86%'], ['9', 'Beyonc', '138338892', '0', '1856', '1.49%'], ['10', 'ene10ta rre neymarjr', '131662459', '1140', '4638', '1.18%'], ['11', 'National Geographic', '129863374', '133', '21748', '0.11%'], ['12', 'Taylor Swift', '125617925', '0', '419', '1.28%'], ['13', 'Justin Bieber', '125388051', '349', '5006', '1.30%'], ['14', 'Kendall', '121435146', '202', '3076', '2.10%'], ['15', 'Barbie', '109738327', '1327', '6026', '0.97%'], ['16', 'Jennifer Lopez', '109581239', '1132', '2747', '1.16%'], ['17', 'Khlo', '103426333', '130', '3713

In [172]:
#convert to a DataFrame
my_DF = pandas.DataFrame(columns = my_head)
for row in my_rows:
    dict_row = dict(zip(my_head,row))
    my_DF.loc[my_rows.index(row)]=dict_row    
my_DF

# Notice that there is an error in Beyonce's account name, which I have checked is a bug from the website itself.

,Rank,Profile,Total Followers,Total Following,Total Posts,Total Influence
0,1,Instagram,327917534,224,6192,0.27%
1,2,Cristiano Ronaldo,198127767,444,2742,3.46%
2,3,Ariana Grande,171750334,999,4477,1.06%
3,4,therock,168535736,342,4740,1.04%
4,5,Selena Gomez,166119117,92,1573,2.82%
5,6,Kylie,158087797,134,6274,4.06%
6,7,Kim Kardashian West,157248175,14,4976,0.91%
7,8,Leo Messi,140974925,227,556,2.86%
8,9,Beyonc,138338892,0,1856,1.49%
9,10,ene10ta rre neymarjr,131662459,1140,4638,1.18%


In [173]:
#data types conversion
for i in range(len(my_DF)):
    my_DF.loc[i,'Total Influence'] = float(my_DF.loc[i,'Total Influence'].strip('%'))/100

my_DF = my_DF.astype({'Rank':'int',
                      'Profile':'str',
                      'Total Followers':'int64',
                      'Total Following':'int64',
                      'Total Posts':'int64',
                      'Total Influence':'float'})
my_DF = my_DF.astype({'Rank':pandas.CategoricalDtype(ordered=True)})
my_DF.dtypes

Rank               category
Profile              object
Total Followers       int64
Total Following       int64
Total Posts           int64
Total Influence     float64
dtype: object

In [174]:
my_DF

,Rank,Profile,Total Followers,Total Following,Total Posts,Total Influence
0,1,Instagram,327917534,224,6192,0.0027
1,2,Cristiano Ronaldo,198127767,444,2742,0.0346
2,3,Ariana Grande,171750334,999,4477,0.0106
3,4,therock,168535736,342,4740,0.0104
4,5,Selena Gomez,166119117,92,1573,0.0282
5,6,Kylie,158087797,134,6274,0.0406
6,7,Kim Kardashian West,157248175,14,4976,0.0091
7,8,Leo Messi,140974925,227,556,0.0286
8,9,Beyonc,138338892,0,1856,0.0149
9,10,ene10ta rre neymarjr,131662459,1140,4638,0.0118



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [133]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [134]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,6
7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,8
9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [135]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [136]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [61]:
import numpy as np
def recursive_spider(root_url, base_url = None, other_links = set(), limit = np.inf):
    #keywords used to define the areas to find hyperlinks 
    root_soup = bs4.BeautifulSoup(requests.get(root_url).text)
    #find all hyperlinks related to celebrities in bbc's programme sites
    results = root_soup.body.find_all('a', href=re.compile(r'/programmes/articles/'), 
                                     string=[re.compile(r'(?:C|c)eleb(?:s|rity|rities)'), re.compile('(?:s|S)tars'), re.compile('ists')])
    if results:
        #since the search is actually within the bbc programmes site, we need to specify "programmes" in href
        for tag in results:
            current_url = urllib.parse.urljoin(base_url, tag.get('href')) if not re.match(r'http', tag.get('href')) else tag.get('href')
            before = len(other_links)
            if len(other_links) == limit:
                break
            else:
                other_links.add((root_url, current_url, tag.text, ','.join(tag.get('class'))))
                #recursive spidering
                #the end-point is set by limit number    
                if len(other_links) == before:
                    break
                else:
                    other_links = recursive_spider(current_url, base_url = base_url, other_links = other_links, limit = limit)
    return other_links

In [62]:
my_root_url = "https://www.bbc.co.uk/programmes/articles/3QcD9W13Dr0bxmt4CMWVkGk/7-stars-who-have-personal-experiences-of-online-bullying"
my_other_links = recursive_spider(my_root_url, base_url = "https://www.bbc.co.uk", limit = 200)
print (my_other_links)

{('http://www.bbc.co.uk/programmes/articles/1dNz6zpmmycMnFWTnskGCmv/things-you-should-never-ask-celebrities', 'http://www.bbc.co.uk/programmes/articles/4KNnWcL3QpFhB08KGnTmbjJ/12-stars-who-took-back-control-of-their-own-lives', '12 stars who took back control of their own lives', 'br-blocklink__link,promotion__link'), ('http://www.bbc.co.uk/programmes/articles/1hykdzPNZTC7qSNNz5wSq2K/8-stars-who-know-the-meaning-of-hard-work', 'http://www.bbc.co.uk/programmes/articles/1hykdzPNZTC7qSNNz5wSq2K/8-stars-who-know-the-meaning-of-hard-work', '8 stars who know the meaning of hard work', 'block-link__overlay-link,promotion__link'), ('http://www.bbc.co.uk/programmes/articles/4kS414VHvYzCz6YMM8qVm43/5-stars-keeping-us-all-in-the-dark-about-their-new-albums', 'http://www.bbc.co.uk/programmes/articles/4kS414VHvYzCz6YMM8qVm43/5-stars-keeping-us-all-in-the-dark-about-their-new-albums', '5 stars keeping us all in the dark about their new albums', 'block-link__overlay-link,promotion__link'), ('https://

In [63]:
my_url_DF = pandas.pandas.DataFrame(columns = ['root','object','words','class'])
for i in range(len(my_other_links)):
    my_url_DF.loc[i] = dict(zip(['root','object','words','class'],list(my_other_links)[i]))
my_url_DF

,root,object,words,class
0,http://www.bbc.co.uk/programmes/articles/1dNz6...,http://www.bbc.co.uk/programmes/articles/4KNnW...,12 stars who took back control of their own lives,"br-blocklink__link,promotion__link"
1,http://www.bbc.co.uk/programmes/articles/1hykd...,http://www.bbc.co.uk/programmes/articles/1hykd...,8 stars who know the meaning of hard work,"block-link__overlay-link,promotion__link"
2,http://www.bbc.co.uk/programmes/articles/4kS41...,http://www.bbc.co.uk/programmes/articles/4kS41...,5 stars keeping us all in the dark about their...,"block-link__overlay-link,promotion__link"
3,https://www.bbc.co.uk/programmes/articles/m0p6...,https://www.bbc.co.uk/programmes/articles/1KqZ...,6 artists who have forgotten their own lyrics,br-blocklink__link
4,http://www.bbc.co.uk/programmes/articles/1dNz6...,http://www.bbc.co.uk/programmes/articles/5xH9w...,Eight celebrity romantic gestures to inspire y...,"br-blocklink__link,promotion__link"
5,https://www.bbc.co.uk/programmes/articles/4hz0...,https://www.bbc.co.uk/programmes/articles/4hz0...,View 6 inspirational stars who have challenged...,block-link__overlay-link
6,https://www.bbc.co.uk/programmes/articles/1KqZ...,https://www.bbc.co.uk/programmes/articles/m0p6...,8 stars who have made powerful statements for ...,br-blocklink__link
7,http://www.bbc.co.uk/programmes/articles/23qQY...,http://www.bbc.co.uk/programmes/articles/1hykd...,8 stars who know the meaning of hard work,"br-blocklink__link,promotion__link"
8,http://www.bbc.co.uk/programmes/articles/3tpJj...,http://www.bbc.co.uk/programmes/articles/1hykd...,8 stars who know the meaning of hard work,"br-blocklink__link,promotion__link"
9,http://www.bbc.co.uk/programmes/articles/18zsJ...,http://www.bbc.co.uk/programmes/articles/5JYY9...,7 stars who regretted their tattoos,"block-link__overlay-link,promotion__link"


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [64]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3467,"posts-type":false,"posts":[{"id":"190305827231","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190305827231","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190305827231\/my-cat-hates-plastic-christmas-trees","type":"photo","date-gmt":"2020-01-17 11:01:17 GMT","date":"Fri, 17 Jan 2020 12:01:17","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1579258877,"format":"html","reblog-key":"FdTENN3Y","slug":"my-cat-hates-plastic-christmas-trees","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"190305827231\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_190305827231\"><iframe id=\"like_iframe_190305827231\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=lolc

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [65]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [67]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [68]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,190305827231,https://66.media.tumblr.com/94c851dc4ae3a8c9ac...,"Fri, 17 Jan 2020 12:01:17","[cat, cats, lol, lolcat, lolcats]",png
1,190304898263,https://66.media.tumblr.com/b63671be9bde5b9c2a...,"Fri, 17 Jan 2020 10:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
2,190295315782,https://66.media.tumblr.com/60a2b6d7356f547032...,"Thu, 16 Jan 2020 22:01:32","[cat, cats, lol, lolcat, lolcats]",jpg
3,190287026776,https://66.media.tumblr.com/6f26a4d5b19774f06c...,"Thu, 16 Jan 2020 10:01:16","[cat, cats, lol, lolcat, lolcats]",png
4,190233110922,https://66.media.tumblr.com/42458742dedc510f3d...,"Mon, 13 Jan 2020 12:03:57","[cat, cats, lol, lolcat, lolcats]",jpg
5,190232075547,https://66.media.tumblr.com/65b1e6efd6ec145872...,"Mon, 13 Jan 2020 10:00:35","[cat, cats, lol, lolcat, lolcats]",jpg
6,190230902981,https://66.media.tumblr.com/616b4faab5a289a369...,"Mon, 13 Jan 2020 08:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
7,190229342226,https://66.media.tumblr.com/28121ad01b05305b89...,"Mon, 13 Jan 2020 06:00:57","[cat, cats, lol, lolcat, lolcats]",jpg
8,190209640041,https://66.media.tumblr.com/9b735740f517f3673b...,"Sun, 12 Jan 2020 06:01:05","[cat, cats, lol, lolcat, lolcats]",jpg
9,190193496849,https://66.media.tumblr.com/1ffa3c880a55f80f62...,"Sat, 11 Jan 2020 10:01:29","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [70]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [71]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [78]:
with open('../data/corpora/gutenberg/shakespeare-hamlet.txt') as f:
    hamlet = f.read()
print(hamlet[-700:])

,
  Hor. Of that I shall haue alwayes cause to speake,
And from his mouth
Whose voyce will draw on more:
But let this same be presently perform'd,
Euen whiles mens mindes are wilde,
Lest more mischance
On plots, and errors happen

   For. Let foure Captaines
Beare Hamlet like a Soldier to the Stage,
For he was likely, had he beene put on
To haue prou'd most royally:
And for his passage,
The Souldiours Musicke, and the rites of Warre
Speake lowdly for him.
Take vp the body; Such a sight as this
Becomes the Field, but heere shewes much amis.
Go, bid the Souldiers shoote.

Exeunt. Marching: after the which, a Peale of Ordenance are shot
off.


FINIS. The tragedie of HAMLET, Prince of Denmarke.



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [79]:
targetDir = '../data/corpora/gutenberg' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and file.name.startswith('shakespeare') and file.name.endswith('txt')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [80]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
shakespeare-caesar.txt,[The Tragedie of Julius Caesar by William Shak...
shakespeare-hamlet.txt,[The Tragedie of Hamlet by William Shakespeare...
shakespeare-macbeth.txt,[The Tragedie of Macbeth by William Shakespear...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [81]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂�!���Wn)Ϗ������a�*?p6�Ƒ2?��N����wӍq�A�ڽ|h��z�;%���pƍg�๑���s�����9��fQ�������$����pw��Շ���W[���f��{�M��M�(�M=gã�����l�?��r��s|Ǐ}׏B��o�:E}����N>b�t����{:�M2��I�����
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;4:���ve�WƟ�0�.�y���V�j������bh��i�)�uW������[�jGnG�T,?9�u]��-1�s��7e��.��XU%�ޗ5��[��²�؅���4w�W��N��淼)��R�MT,�,F4��o>n
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@RQ`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [6]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [82]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [85]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [87]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [88]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [89]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [90]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [111]:
#extraction from raw text
senti_records = []
with open("../data/corpora/sentiwordnet/SentiWordNet_3.0.0.txt") as file:
     for line in file:
            if not line.strip().startswith('#'):
                senti_records.append(line.split(maxsplit = 5))
senti_records

[['a',
  '00001740',
  '0.125',
  '0',
  'able#1',
  '(usually followed by `to\') having the necessary means or skill or know-how or authority to do something; "able to swim"; "she was able to program her computer"; "we were at last able to buy a car"; "able to get a grant for the project"\n'],
 ['a',
  '00002098',
  '0',
  '0.75',
  'unable#1',
  '(usually followed by `to\') not having the necessary means or skill or know-how; "unable to get to town without a car"; "unable to obtain funds"\n'],
 ['a',
  '00002312',
  '0',
  '0',
  'dorsal#2',
  'abaxial#1\tfacing away from the axis of an organ or organism; "the abaxial surface of a leaf is the underside or side facing away from the stem"\n'],
 ['a',
  '00002527',
  '0',
  '0',
  'ventral#2',
  'adaxial#1\tnearest to or facing toward the axis of an organ or organism; "the upper side of a leaf is known as the adaxial surface"\n'],
 ['a',
  '00002730',
  '0',
  '0',
  'acroscopic#1',
  'facing or on the side toward the apex\n'],
 ['a',
 

In [113]:
senti_DF = pandas.DataFrame(data ={"POS":[x[0] for x in senti_records],
                             "ID":[x[1] for x in senti_records],
                             "PosScore": [x[2] for x in senti_records],
                             "NegScore": [x[3] for x in senti_records],
                             "SynsetTerms": [x[4] for x in senti_records],
                             "Gloss": [x[5] for x in senti_records]})
senti_DF

,POS,ID,PosScore,NegScore,SynsetTerms,Gloss
0,a,00001740,0.125,0,able#1,(usually followed by `to') having the necessar...
1,a,00002098,0,0.75,unable#1,(usually followed by `to') not having the nece...
2,a,00002312,0,0,dorsal#2,abaxial#1\tfacing away from the axis of an org...
3,a,00002527,0,0,ventral#2,adaxial#1\tnearest to or facing toward the axi...
4,a,00002730,0,0,acroscopic#1,facing or on the side toward the apex\n
5,a,00002843,0,0,basiscopic#1,facing or on the side toward the base\n
6,a,00002956,0,0,abducting#1,abducent#1\tespecially of muscles; drawing awa...
7,a,00003131,0,0,adductive#1,adducting#1 adducent#1\tespecially of muscles;...
8,a,00003356,0,0,nascent#1,"being born or beginning; ""the nascent chicks"";..."
9,a,00003553,0,0,emerging#2,"emergent#2\tcoming into existence; ""an emergen..."


In [7]:
#extraction from raw PDF
pdf_url = "https://arxiv.org/pdf/1508.06257.pdf"
pdf_request = requests.get(pdf_url, stream = True)
pdf_Bytes = io.BytesIO(pdf_request.content)
pdf_content = readPDF(pdf_Bytes)
print (pdf_content)

Prediction of Cyberbullying Incidents on the Instagram

Social Network

Homa Hosseinmardi‡, Sabrina Arredondo Mattson§, Rahat Ibn Raﬁq‡,

Richard Han‡, Qin Lv‡, Shivakant Mishra‡

‡Computer Science Department, §Institute of Behavioral Science

University of Colorado Boulder, Boulder, CO 80309 USA

homa.hosseinmardi@colorado.edu

5
1
0
2

 

g
u
A
5
2

 

 
 
]

R

I
.
s
c
[
 
 

1
v
7
5
2
6
0

.

8
0
5
1
:
v
i
X
r
a

ABSTRACT
Cyberbullying is a growing problem affecting more than half
of all American teens. The main goal of this paper is to in-
vestigate fundamentally new approaches to understand and
automatically detect and predict incidents of cyberbullying
in Instagram, a media-based mobile social network. In this
work, we have collected a sample data set consisting of In-
stagram images and their associated comments. We then de-
signed a labeling study and employed human contributors at
the crowd-sourced CrowdFlower website to label these me-
dia sessions for cyberbullying. A detai

In [25]:
pdf_comps = {}
temp = re.split(r'\n\d\.[ \n]',pdf_content)
#get abstract
pdf_comps['ABSTRACT'] = re.findall(r'(?:Abstract|ABSTRACT)(?:.|\n)*',re.sub(r'\n',' ', temp[0]))[0]
print (pdf_comps['ABSTRACT'])

ABSTRACT Cyberbullying is a growing problem affecting more than half of all American teens. The main goal of this paper is to in- vestigate fundamentally new approaches to understand and automatically detect and predict incidents of cyberbullying in Instagram, a media-based mobile social network. In this work, we have collected a sample data set consisting of In- stagram images and their associated comments. We then de- signed a labeling study and employed human contributors at the crowd-sourced CrowdFlower website to label these me- dia sessions for cyberbullying. A detailed analysis of the labeled data is then presented, including a study of relation- ships between cyberbullying and a host of features such as cyberaggression, profanity, social graph features, temporal commenting behavior, linguistic content, and image content. Using the labeled data, we further design and evaluate the performance of classiﬁers to automatically detect and pre- dict incidents of cyberbullying and cyber

In [26]:
#get title, it is hard to distinguish the title from authors without prior knowledge,the method here only appliesto titles without commas
head = re.sub(r'\n\n','\n', temp[0])
title = ''
for line in head.split("\n"):
    if "," not in line:
        title += (line+" ")
    else:
        break
pdf_comps['TITLE'] = title
print (pdf_comps['TITLE'])

Prediction of Cyberbullying Incidents on the Instagram Social Network 


In [27]:
head = re.sub(r'\n',' ', head)
pdf_comps['AUTHORS'] = re.findall(r'(?:.|\n)*.edu',head.replace(title, ""))[0]
print (pdf_comps['AUTHORS'])

Homa Hosseinmardi‡, Sabrina Arredondo Mattson§, Rahat Ibn Raﬁq‡, Richard Han‡, Qin Lv‡, Shivakant Mishra‡ ‡Computer Science Department, §Institute of Behavioral Science University of Colorado Boulder, Boulder, CO 80309 USA homa.hosseinmardi@colorado.edu


In [28]:
#get other parts
for i in range(1,len(temp)):
    part = temp[i].strip()
    part_name = part.split('\n')[0]
    if re.findall(r'[a-z]',part_name):
        # try to exclude some special cases
        pdf_comps[temp[i-1].strip().split('\n')[0]] += part         
    else:
        pdf_comps[part_name] = part 

In [29]:
pdf_DF = pandas.DataFrame(columns = ['component', 'content'])
for i in range(len(pdf_comps.keys())):
    k = list(pdf_comps.keys())[i]
    pdf_DF.loc[i,'component'] = k 
    pdf_DF.loc[i,'content'] = pdf_comps[k]
pdf_DF

,component,content
0,ABSTRACT,ABSTRACT Cyberbullying is a growing problem af...
1,TITLE,Prediction of Cyberbullying Incidents on the I...
2,AUTHORS,"Homa Hosseinmardi‡, Sabrina Arredondo Mattson§..."
3,INTRODUCTION,INTRODUCTION\n\nAs online social networks (OSN...
4,RELATED WORKS,RELATED WORKS\n\nPrior works that investigated...
5,DATA COLLECTION,DATA COLLECTION\n\nStarting from a random seed...
6,CYBERBULLYING LABELING,"CYBERBULLYING LABELING\n\nIn this section, we ..."
7,ANALYSIS AND CHARACTERIZATION,ANALYSIS AND CHARACTERIZATION\n\nOF GROUND TRU...
8,CYBERBULLYING DETECTION,CYBERBULLYING DETECTION\n\nBased on the labele...
9,CYBERBULLYING PREDICTION,CYBERBULLYING PREDICTION\n\nOur next goal is t...


In [3]:
#extraction from Word
word_url = "https://www.cde.ca.gov/ls/ss/se/documents/bullymodule1.docx"
word_request = requests.get(word_url, stream=True)
my_word = docx.Document(io.BytesIO(word_request.content))
my_word

In [45]:
#get author and title information, which is consistent with the file property instead of being extracted from text
word_DF = pandas.DataFrame(columns = ['component', 'content'])
word_DF.loc[len(word_DF)] = {'component':'title','content':my_word.core_properties.title}
word_DF.loc[len(word_DF)] = {'component':'author','content':my_word.core_properties.author}
word_DF

,component,content
0,title,Bullying Module - Bullying and School Safety (...
1,author,Nancy Zarenda


In [46]:
comp_indice = []
for i in range(len(my_word.paragraphs)):
    match = re.findall(r'(?:Chapter|Section) \d:', my_word.paragraphs[i].text)
    if match:
        comp_indice.append((match[0], i))
        if len(comp_indice)>1:
            start = comp_indice[-2][1]
            word_DF.loc[len(word_DF)] = {'component':comp_indice[-2][0],
                                         'content':'\n'.join([x.text.strip() for x in my_word.paragraphs[start:i]])}
#Adding the last chapter
word_DF.loc[len(word_DF)] = {'component':comp_indice[-1][0], 
                             'content':'\n'.join([x.text.strip() for x in my_word.paragraphs[comp_indice[-1][1]:len(my_word.paragraphs)]])}
word_DF

,component,content
0,title,Bullying Module - Bullying and School Safety (...
1,author,Nancy Zarenda
2,Chapter 1:,Chapter 1: What Is Bullying?\nBullying is an u...
3,Chapter 2:,Chapter 2: What Is Cyberbullying and How Can W...
4,Chapter 3:,Chapter 3: What Roles Do Students Play?\nThere...
5,Chapter 4:,Chapter 4: Where Does Bullying Occur?\nBullyin...
6,Chapter 5:,Chapter 5: How Do We Prevent Bullying?\nInclud...
7,Chapter 6:,Chapter 6: How Do We Respond to Bullying?\nInt...
8,Chapter 7:,Chapter 7: Bullying and Students with Disabili...
9,Chapter 8:,"Chapter 8: Bullying and Lesbian, Gay, Bisexual..."


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

